In [1]:
# !pip install transformers datasets evaluate scikit-learn torch

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# import kagglehub
# waalbannyantudre_hate_speech_detection_curated_dataset_path = kagglehub.dataset_download('waalbannyantudre/hate-speech-detection-curated-dataset')

print('Data source import complete.')

import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.layers import  Input,Dense,Embedding, LSTM,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.metrics import accuracy_score
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from nltk.corpus import stopwords
import nltk
# nltk.download('stopwords')


Data source import complete.


In [6]:
dataset_path = "/kaggle/input/hate-speech-detection-curated-dataset/HateSpeechDatasetBalanced.csv"
df = pd.read_csv(dataset_path)
df.head()

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1


DATA CLEANING

In [7]:

stop_words = set(stopwords.words('english'))

#Text Preprocessing Function
def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

df['CleanedContent'] = df['Content'].apply(clean_text)


TEST TRAIN SPLIT

In [9]:
#LIMITED DF
limited_df = df.sample(n=200, random_state=42)
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    limited_df['CleanedContent'].values,
    limited_df['Label'].values,
    test_size=0.2,
    random_state=42
)

# X_train_texts, X_test_texts, y_train, y_test = train_test_split(
#     df['CleanedContent'].values,
#     df['Label'].values,
#     test_size=0.2,
#     random_state=42
# )

print("Training set size:", len(X_train_texts))
print("Test set size:", len(X_test_texts))

Training set size: 160
Test set size: 40


HUGGINGFACE TRANFORMERS

In [10]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# Load pretrained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTr

PRE PROCCESSING

In [11]:
# Tokenize texts (use max_length=512 for typical BERT-based models)
train_encodings = tokenizer(X_train_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors="tf")
test_encodings = tokenizer(X_test_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors="tf")


In [12]:
import tensorflow as tf

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(128)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(128)


In [23]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Compile the model
# optimizer = Adam(learning_rate=5e-5)
# print(optimizer)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer='adam', loss=loss, metrics=metrics)



TRAINING

In [24]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)


Epoch 1/5
2/2 [==============================] - 183s 48s/step - loss: 0.7150 - accuracy: 0.3875 - val_loss: 0.6363 - val_accuracy: 0.6750
Epoch 2/5
2/2 [==============================] - 145s 40s/step - loss: 0.8860 - accuracy: 0.3938 - val_loss: 0.7106 - val_accuracy: 0.6750
Epoch 3/5
2/2 [==============================] - 142s 40s/step - loss: 0.6983 - accuracy: 0.4125 - val_loss: 0.6980 - val_accuracy: 0.6750
Epoch 4/5
2/2 [==============================] - 142s 39s/step - loss: 0.6932 - accuracy: 0.4187 - val_loss: 0.6885 - val_accuracy: 0.6750
Epoch 5/5
2/2 [==============================] - 142s 39s/step - loss: 0.6985 - accuracy: 0.4187 - val_loss: 0.6994 - val_accuracy: 0.6750


PREDICTIONS

In [23]:
y_pred_logits = model.predict(test_dataset).logits
y_pred_probs = tf.sigmoid(y_pred_logits).numpy().squeeze()

# Convert probabilities to labels (threshold > 0.5)
y_pred_labels = (y_pred_probs > 0.5).astype(int)



2/2 [==============================] - 15s 3s/step


In [36]:
import random

# Test a sentence
for i in range (20):
  num = random.randint(1, len(X_test_texts))
  print(X_test_texts[num],f"Prob: {y_pred_probs[num]:.4f} -> Label: {y_pred_labels[num]}")


protection expire three months end time left wikipedia alone created additional accounts problems maybe someone open hearing say since one going unblock merely wasting time Prob: 0.5946 -> Label: 1
note click line might need clean obvious intent line Prob: 0.5946 -> Label: 1
censorship article constantly censored zionist hawks supposed even handed analysis phony propagandists like jay jg espouse bullshit views contrary views international community via un human rights groups etc etc ruining wikipedia turning another arm israel lobby criticism deemed conspiracy theory ruining everything site supposed stand fucking nazis Prob: 0.5946 -> Label: 1
love https www linkedin com pub wayne weber got air force went straight head hk wow Prob: 0.5946 -> Label: 1
love https www linkedin com pub wayne weber got air force went straight head hk wow Prob: 0.5946 -> Label: 1
znali itk dobro doctor squad exertion jude priporuci za license bill biam roditelji galvani cell pivot Prob: 0.5946 -> Label: 1
af

GETTING TOXICITY SCORES

In [38]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
import numpy as np

# Load tokenizer and model
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [40]:
def get_toxicity(text):
    # Tokenize
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)

    # Predict logits
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply sigmoid to get probabilities
    probs = tf.sigmoid(logits)[0].numpy()

    # The model has multiple classes: toxic, severe toxic, obscene, threat, insult, identity hate
    labels = [
        "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
    ]

    # Convert to % and return as dict
    toxicity_scores = {label: round(prob * 100, 2) for label, prob in zip(labels, probs)}
    return toxicity_scores


In [44]:

import random

# Test a sentence
for i in range (20):
  num = random.randint(1, len(X_test_texts))
  scores = get_toxicity(X_test_texts[num])
  print("\n",X_test_texts[num])
  formatted_scores = {k: round(float(v), 2) for k, v in scores.items()}
  for label, score in formatted_scores.items():
      print(f"{label.title()}: {score}%")


# print("Toxicity Levels (%):", scores)



 wrong way clear
Toxic: 0.09%
Severe_Toxic: 0.01%
Obscene: 0.02%
Threat: 0.01%
Insult: 0.02%
Identity_Hate: 0.01%

 general point view comments
Toxic: 0.06%
Severe_Toxic: 0.01%
Obscene: 0.02%
Threat: 0.01%
Insult: 0.02%
Identity_Hate: 0.01%

 women simply whores detest race
Toxic: 97.0%
Severe_Toxic: 5.5%
Obscene: 37.72%
Threat: 1.0%
Insult: 42.36%
Identity_Hate: 65.76%

 mls outrage listen conference xbox next gen fifa article screwed fix information technology operating theater logging away
Toxic: 0.12%
Severe_Toxic: 0.01%
Obscene: 0.02%
Threat: 0.01%
Insult: 0.02%
Identity_Hate: 0.01%

 blacks unemployed
Toxic: 90.77%
Severe_Toxic: 3.61%
Obscene: 10.85%
Threat: 0.82%
Insult: 27.86%
Identity_Hate: 73.69%

 eyes shit goes muslim
Toxic: 98.67%
Severe_Toxic: 20.5%
Obscene: 88.77%
Threat: 2.57%
Insult: 66.85%
Identity_Hate: 73.31%

 someone stole presentation work demonsofvine funny reaction relatable lol highlight
Toxic: 1.71%
Severe_Toxic: 0.01%
Obscene: 0.07%
Threat: 0.01%
Insult: 0.